# Measure 4: High-Aesthetics Network Graph

## Goal
Create a visually stunning social network graph for *Anna Karenina* suitable for a professional presentation.

## Visual Style
* **Palette:** 'Plasma' gradient (Warm colors for central characters, cool for peripheral).
* **Edges:** Curved lines with variable thickness based on interaction strength.
* **Layout:** Spaced out force-directed layout to avoid clutter.

In [ ]:
%pip install networkx pandas matplotlib nltk numpy

In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# --- 1. SETUP ---
nltk.download('punkt')
nltk.download('punkt_tab')

DATA_DIR = '../data'
RESULTS_DIR = '../results'

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

# ANNA KARENINA CONFIG ONLY
CONFIG = {
    "filename": "The Project Gutenberg eBook of Anna Karenina, by Leo Tolstoy.txt",
    "characters": ["Anna", "Vronsky", "Levin", "Kitty", "Karenin", "Stiva", "Dolly", "Betsy"]
}

## 2. Processing Logic

In [ ]:
def load_text(filename):
    filepath = os.path.join(DATA_DIR, filename)
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"ERROR: Could not find {filepath}")
        return ""

def build_graph(text, characters):
    sentences = sent_tokenize(text)
    G = nx.Graph()
    G.add_nodes_from(characters)
    
    char_map = {c.lower(): c for c in characters}
    
    print(f"Scanning {len(sentences)} sentences for interactions...")
    
    for sent in sentences:
        tokens = set(word_tokenize(sent.lower()))
        found = [char_map[c] for c in char_map if c in tokens]
        
        if len(found) > 1:
            for pair in itertools.combinations(found, 2):
                u, v = pair
                if G.has_edge(u, v):
                    G[u][v]['weight'] += 1
                else:
                    G.add_edge(u, v, weight=1)
    return G

## 3. The "Beautiful" Visualizer
This function applies advanced styling: curved edges, gradient colors, and professional typography.

In [ ]:
def draw_beautiful_network(G):
    plt.figure(figsize=(14, 10), facecolor='white')
    ax = plt.gca()
    
    # 1. Metrics for styling
    centrality = nx.degree_centrality(G)
    node_sizes = [v * 8000 + 500 for v in centrality.values()] # Base size + scaling
    
    # 2. Layout (Physics Simulation)
    # k=1.5 pushes nodes apart more, iterations=50 gives it time to settle
    pos = nx.spring_layout(G, k=1.5, iterations=50, seed=42) 
    
    # 3. Draw Edges (Curved & Transparent)
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    max_weight = max(weights) if weights else 1
    
    # Draw each edge with variable thickness and curve
    for (u, v, d) in G.edges(data=True):
        width = (d['weight'] / max_weight) * 4 + 0.5
        # Use a connectionstyle to create the 'curve' effect
        nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], width=width, alpha=0.3, edge_color="#555555", 
                               connectionstyle="arc3,rad=0.1", ax=ax)

    # 4. Draw Nodes (Gradient Color)
    # Create a color map based on centrality
    node_colors = list(centrality.values())
    nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, 
                                   cmap=plt.cm.plasma, alpha=0.9, edgecolors='white', linewidths=2, ax=ax)
    
    # 5. Draw Labels (With white 'halo' outline for readability)
    labels = nx.draw_networkx_labels(G, pos, font_size=12, font_family="sans-serif", font_weight="bold")
    
    # Apply white outline to text
    import matplotlib.patheffects as path_effects
    for _, label in labels.items():
        label.set_path_effects([path_effects.withStroke(linewidth=3, foreground='white')])

    # 6. Final Touches
    plt.title("Character Interaction Network: Anna Karenina", fontsize=18, fontweight='bold', pad=20)
    plt.axis('off')
    
    # Add a colorbar to explain the node colors (Optional, but looks pro)
    # cbar = plt.colorbar(nodes, shrink=0.5, aspect=20, label='Centrality Score')
    # cbar.outline.set_visible(False)

    save_path = f"{RESULTS_DIR}/anna_karenina_beautiful_network.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Beautiful graph saved to: {save_path}")
    plt.show()

## 4. Execution

In [ ]:
def run_beautiful_graph():
    print("Loading text...")
    text = load_text(CONFIG['filename'])
    
    if text:
        G = build_graph(text, CONFIG['characters'])
        if G.number_of_edges() > 0:
            print("Generating high-quality visualization...")
            draw_beautiful_network(G)
        else:
            print("No connections found.")
    else:
        print("File not found.")

run_beautiful_graph()